this notebook is aimed to reorganized prediction data,specificly:
1. find the optimized PCs that should be ommited form prediction
2. constrain the bootstrap procedure according to the standard prediction pipeline 

# Data preparision

In [69]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns

from algo import *

from sklearn.decomposition import PCA
from sklearn.preprocessing import scale
from sklearn.model_selection import LeaveOneOut,GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

import nibabel as nib

In [70]:
# define data path
path = '/nfs/s2/userhome/yanganmin/workingdir/attention_data_complete/train_test_data'

In [71]:
# read fa train and test data, also test on sa
train_X_fa = np.load(os.path.join(path,'train_X_fa.npy'))
test_X_fa = np.load(os.path.join(path,'test_X_fa.npy'))
train_Y_fa = np.load(os.path.join(path,'train_Y_fa.npy'))
test_Y_fa = np.load(os.path.join(path,'test_Y_fa.npy'))
X_sa = np.load(os.path.join(path,'X_sa.npy'))
Y_sa = np.load(os.path.join(path,'Y_sa.npy'))

mask_NaN = np.load(os.path.join(path,'mask_NaN.npy'))

# PCA of X 

In [72]:
# scale data
train_X_fa = scale(train_X_fa)
test_X_fa = scale(test_X_fa)
X_sa = scale(X_sa)

# reduce dimention with PCA
pca = PCA() # reduce dimentions to the number of observations
pca.fit(train_X_fa)
train_X_fa_pc = pca.transform(train_X_fa)
test_X_fa_pc = pca.transform(test_X_fa)
X_sa_pc = pca.transform(X_sa)

print(f'The number of PCs is {train_X_fa_pc.shape[1]}, which is also the number of observations in train_X_fa')

The number of PCs is 314, which is also the number of observations in train_X_fa


# Grid search for the best hyper-Parameter
the best c in Logistic regression using L1 penalty, based on train_X_fa

In [32]:
tuned_parameters = [{'C': np.arange(0.05,1.05,0.05)}]

clf = GridSearchCV(LogisticRegression(penalty='l1',solver='liblinear'),tuned_parameters,cv=10,scoring='f1')
clf.fit(train_X_fa_pc, train_Y_fa)

C_best = clf.best_params_['C']

print(f'Best hyper-Parameter C is {C_best}.')

Best hyper-Parameter C is 0.05.


In [33]:
clf = LogisticRegression(penalty='l1',solver='liblinear',C=C_best) # C value determined by grid search
loo = LeaveOneOut()
LR_beta = []

# Train LR to find PCs with low beta 

In [34]:
for train_index, test_index in loo.split(train_X_fa):
    clf.fit(train_X_fa_pc[train_index], train_Y_fa[train_index])
    LR_beta.append(clf.coef_[0])

LR_beta_matrix = np.zeros(len(LR_beta[0]))
for array in LR_beta:
    LR_beta_matrix = np.vstack((LR_beta_matrix,array))
LR_beta_matrix = LR_beta_matrix[1:,:]
LR_beta_mean = LR_beta_matrix.mean(axis=0)
print(f'the shape of the LR_beta_mean is {LR_beta_mean.shape}.')

the shape of the LR_beta_mean is (314,).


# Gradually rule out PCs with low beta 
for-loop to dermine the best cutoff for beta

In [35]:
PC_record = {} # dictionary to store predicition accuracy with each phase of reduction of PCs
num_PC = []
for beta_cutoff in np.arange(0,np.max(abs(LR_beta_mean)),0.001):
    mask_beta = (abs(LR_beta_mean) > beta_cutoff)
    train_X_fa_masked = train_X_fa_pc[:,mask_beta]
    test_X_fa_masked = test_X_fa_pc[:,mask_beta]
    
    clf.fit(train_X_fa_masked,train_Y_fa)
    
    num_PC_temp = np.sum(mask_beta)
    num_PC.append(num_PC_temp)
    acc = clf.score(test_X_fa_masked,test_Y_fa)
    PC_record[str(beta_cutoff)] = acc

In [36]:
pc_pd = pd.DataFrame({'beta_cutoff':PC_record.keys(),
                     'accuracy':PC_record.values(),
                     'num_PC':num_PC})

In [37]:
pc_pd.head()

,beta_cutoff,accuracy,num_PC
0,0.0,0.955128,120
1,0.001,0.955128,64
2,0.002,0.961538,54
3,0.003,0.961538,48
4,0.004,0.955128,46


In [38]:
save_path = '/nfs/s2/userhome/yanganmin/workingdir/attention_reorganized_results/'
pc_pd.to_csv(os.path.join(save_path,'pc_selection','fa_pc_selection.csv'))

In [39]:
pc_pd[pc_pd['accuracy'] == pc_pd.max()['accuracy']]

,beta_cutoff,accuracy,num_PC
10,0.01,0.974359,25
11,0.011,0.974359,24
12,0.012,0.974359,23


In [40]:
beta_cutoff_optimized = 0.01 # let beta cutoff be 0.01 

# employ new X with mask of optimized beta cutoff

In [41]:
mask_beta_optimized = (abs(LR_beta_mean) > beta_cutoff_optimized)
train_X_fa_optimized = train_X_fa_pc[:,mask_beta_optimized]
test_X_fa_optimized = test_X_fa_pc[:,mask_beta_optimized]
X_sa_optimized = X_sa_pc[:,mask_beta_optimized]

In [42]:
# save beta cutoff mask 
np.save(os.path.join(save_path,'pc_selection','train_fa_pc_mask.npy'),mask_beta_optimized)

# Train Logistic Regression

In [43]:
clf = LogisticRegression(penalty='l1',solver='liblinear',C=C_best) # C value determined by grid search

beta_pool = []
loo = LeaveOneOut()
leave_out_result = [] # tupple inside (train_accuracy,val_accuracy,test_accuracy)
sa_result = []

for train_index, test_index in loo.split(train_X_fa_optimized):
    clf.fit(train_X_fa_optimized[train_index], train_Y_fa[train_index])

    coef = clf.coef_[0]
    coef_modified = mask_back(mask_beta,coef,mask_type='beta')
    beta_fa = pca.inverse_transform(coef_modified)
    beta_pool.append(beta_fa)

    val_accuracy =  clf.score(train_X_fa_optimized[test_index], train_Y_fa[test_index])
    train_accuracy = clf.score(train_X_fa_optimized[train_index],train_Y_fa[train_index])
    test_accuracy = clf.score(test_X_fa_optimized,test_Y_fa)
    leave_out_result.append((train_accuracy,val_accuracy,test_accuracy))

    sa_result.append(clf.score(X_sa_optimized,Y_sa))

# predict accuracy
train_acc_array = [i[0] for i in leave_out_result]
val_acc_array = [i[1] for i in leave_out_result]
test_acc_array = [i[2] for i in leave_out_result]

# save prediction results
train_result = np.array(train_acc_array)
val_result = np.array(val_acc_array)
test_result = np.array(test_acc_array)
sa_result = np.array(sa_result)
result_matrix = np.c_[train_result,val_result,test_result,sa_result]
df_result = pd.DataFrame(result_matrix, columns=['train','val','test','sa'])

In [44]:
df_result.head()

,train,val,test,sa
0,0.99361,1.0,0.974359,0.651064
1,0.99361,1.0,0.974359,0.651064
2,0.99361,1.0,0.974359,0.651064
3,0.99361,1.0,0.974359,0.651064
4,0.99361,1.0,0.974359,0.651064


In [45]:
df_result.to_csv(os.path.join(save_path,'predict_results','train_fa.csv'))

# Average beta value

In [46]:
avg_beta = np.zeros(beta_pool[0].shape[0])

for pool in beta_pool:
    avg_beta = np.vstack((avg_beta,pool))
avg_beta = avg_beta[1:,:]
avg_beta = avg_beta.mean(axis=0)

In [47]:
# back-project beta map to mni space
coef_mni = mask_back(mask_NaN,avg_beta,mask_type='mni')
coef_mni = np.nan_to_num(coef_mni) # convert nan to zero
nif_beta_raw = to_mni(coef_mni)
nib.save(nif_beta_raw, os.path.join(save_path,'beta','fa_beta_nan_transferred.nii.gz'))

# Bootstrap for beta distribution 

In [50]:
import time 

sample_size = 1000
boot_coef = np.zeros(mask_NaN.shape[0])
concat_data = np.c_[train_X_fa,train_Y_fa]

In [25]:
full_start = time.time()

# the bootstrap procedure should follow the aforementioned scheme  
for iteration in range(sample_size):
    start = time.time()
    
    bootstraped_data = bootstrap_data(concat_data)
    X = bootstraped_data[:,:-1] 
    Y = bootstraped_data[:,-1]           
    
    # average beta 
    LR_beta = []
    for train_index, test_index in loo.split(X):
        clf.fit(X[train_index], Y[train_index])
        LR_beta.append(clf.coef_[0])

    LR_beta_matrix = np.zeros(len(LR_beta[0]))
    for array in LR_beta:
        LR_beta_matrix = np.vstack((LR_beta_matrix,array))
    LR_beta_matrix = LR_beta_matrix[1:,:]
    LR_beta_mean = LR_beta_matrix.mean(axis=0)
    
    # determine PCs to be ruled 
    beta_cutoff_pool = np.arange(0,np.max(abs(LR_beta_mean)),0.01)
    acc_pool = []
    for beta_cutoff in np.arange(0,np.max(abs(LR_beta_mean)),0.01):
        mask_beta = (abs(LR_beta_mean) > beta_cutoff)
        train_X_fa_masked = X[:,mask_beta]
        test_X_fa_masked = test_X_fa_pc[:,mask_beta]
    
        clf.fit(train_X_fa_masked,Y)
    
        acc = clf.score(test_X_fa_masked,test_Y_fa)
        acc_pool.append(acc)
    
    index = acc_pool.argmax()
    beta_cutoff = beta_cutoff_pool[index]
    
    mask_beta = (abs(LR_beta_mean) > beta_cutoff)
    train_X_fa_masked = X[:,mask_beta]
    # now the PCs that should be reduced according to the newly bootstraped data distribution is determined 
    
    clf.fit(train_X_fa_masked,Y)
    coef = clf.coef_[0]
    coef_modified = mask_back(mask_beta,coef,mask_type='beta')
    inverse_coef= pca.inverse_transform(coef_modified)
    coef_mni = mask_back(mask_NaN,inverse_coef,mask_type='mni')
    boot_coef = np.vstack((boot_coef,coef_mni))
    
    end = time.time()
    print(f'This is iteration {iteration}, using time {end-start}s.')
boot_coef = boot_coef[1:,:]

full_end = time.time()
print(f'The total time cost is {full_end-full_start}s.')

# save bootstrap_1000 distribution 
np.save(os.path.join(save_path,'bootstrap','train_fa_bootstrap.npy'),boot_coef)

IndexError: boolean index did not match indexed array along dimension 1; dimension is 314 but corresponding boolean dimension is 149270